In [1]:
import pandas as pd

# Read the three Parquet files
df1 = pd.read_parquet('/content/Dec2018-LabsCUR-00001.snappy.parquet')
df2 = pd.read_parquet('/content/Nov2018-LabsCUR-00001.snappy.parquet')
df3 = pd.read_parquet('/content/Oct2018-LabsCUR-00001.snappy.parquet')

# Concatenate the DataFrames
# Now `df` contains all the data from the three Parquet files
df = pd.concat([df1, df2, df3], ignore_index=True)

effective_cur_columns =['bill_payer_account_id','bill_billing_period_start_date','bill_billing_period_end_date','line_item_usage_account_id', 'line_item_line_item_type', 'line_item_usage_start_date', 'line_item_usage_end_date',
               'line_item_product_code','product_product_name', 'product_instance_family', 'product_instance_type', 'product_instance_type_family','line_item_usage_type', 'line_item_operation', 'line_item_availability_zone',
               'line_item_usage_amount','line_item_normalization_factor', 'line_item_normalized_usage_amount','line_item_unblended_rate', 'line_item_unblended_cost', 'pricing_term', 'pricing_unit','pricing_offering_class', 'pricing_purchase_option',
               'reservation_amortized_upfront_cost_for_usage', 'reservation_amortized_upfront_fee_for_billing_period', 'reservation_effective_cost', 'reservation_end_time','reservation_normalized_units_per_reservation',
               'reservation_number_of_reservations', 'reservation_recurring_fee_for_usage', 'reservation_reservation_a_r_n', 'reservation_start_time', 'reservation_subscription_id', 'reservation_total_reserved_normalized_units',
               'reservation_total_reserved_units', 'reservation_units_per_reservation', 'reservation_unused_amortized_upfront_fee_for_billing_period', 'reservation_unused_normalized_unit_quantity', 'reservation_unused_quantity',
               'reservation_unused_recurring_fee', 'reservation_upfront_value']
# projection on the df to get data of only selected colmns
df = df.loc[:,effective_cur_columns]

# filtering df for only selected product names
product_names = [
    'Amazon Elastic Compute Cloud',
    'Amazon Relational Database Service',
    'Amazon ElastiCache',
    'Amazon OpenSearch Service',
    'Amazon Elasticsearch Service',
    'Amazon Redshift',
    'Amazon DynamoDB'
]

df = df[df['product_product_name'].isin(product_names)]

df



,bill_payer_account_id,bill_billing_period_start_date,bill_billing_period_end_date,line_item_usage_account_id,line_item_line_item_type,line_item_usage_start_date,line_item_usage_end_date,line_item_product_code,product_product_name,product_instance_family,...,reservation_start_time,reservation_subscription_id,reservation_total_reserved_normalized_units,reservation_total_reserved_units,reservation_units_per_reservation,reservation_unused_amortized_upfront_fee_for_billing_period,reservation_unused_normalized_unit_quantity,reservation_unused_quantity,reservation_unused_recurring_fee,reservation_upfront_value
0,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,2018-04-24T00:34:01.000Z,1141706146,2976.00000000000,744.0000000000,744.0000000000,22.336986,2976.0,744.0,22.32,263.0
1,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,2018-04-03T01:21:13.000Z,1088703662,5952.00000000000,1488.0000000000,744.0000000000,0.000000,0.0,0.0,0.00,446.0
2,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,2018-04-03T01:21:13.000Z,1088703646,2976.00000000000,744.0000000000,744.0000000000,0.150457,8.0,2.0,0.15,659.0
3,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,2018-08-13T18:13:55.000Z,1411573020,372.00000000000,744.0000000000,744.0000000000,0.000000,0.0,0.0,0.00,0.0
4,836060457634,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,836060457634,RIFee,2018-12-01 00:00:00+00:00,2018-12-31 23:59:59+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,2018-04-03T01:21:13.000Z,1088703637,1116.00000000000,2232.0000000000,744.0000000000,0.000000,0.0,0.0,0.00,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334887,836060457634,2018-10-01 00:00:00+00:00,2018-11-01 00:00:00+00:00,011254002932,Usage,2018-10-31 23:00:00+00:00,2018-11-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,,1174267096,,,,0.000000,0.0,0.0,0.00,0.0
334888,836060457634,2018-10-01 00:00:00+00:00,2018-11-01 00:00:00+00:00,011254002932,Usage,2018-10-31 23:00:00+00:00,2018-11-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,,1174267096,,,,0.000000,0.0,0.0,0.00,0.0
334889,836060457634,2018-10-01 00:00:00+00:00,2018-11-01 00:00:00+00:00,011254002932,Usage,2018-10-31 23:00:00+00:00,2018-11-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,,1174267096,,,,0.000000,0.0,0.0,0.00,0.0
334890,836060457634,2018-10-01 00:00:00+00:00,2018-11-01 00:00:00+00:00,011254002932,Usage,2018-10-31 23:00:00+00:00,2018-11-01 00:00:00+00:00,AmazonEC2,Amazon Elastic Compute Cloud,,...,,1174267096,,,,0.000000,0.0,0.0,0.00,0.0


In [2]:
# getting DiscountedUsage line_item_line_item_type to fetch pricing_offering_class, pricing_purchase_option, reservation_reservation_a_r_n
# Filter the DataFrame where 'line_item_line_item_type' is 'DiscountedUsage'
discounted_usage_df = df[df['line_item_line_item_type'] == 'DiscountedUsage']

# Group by the required columns and get the unique combinations
discounted_usage_df = discounted_usage_df[['pricing_offering_class', 'pricing_purchase_option', 'reservation_reservation_a_r_n']].drop_duplicates()

discounted_usage_df


,pricing_offering_class,pricing_purchase_option,reservation_reservation_a_r_n
13,standard,No Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28164,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28166,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28167,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28168,standard,No Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28169,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28170,standard,Partial Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...
28171,convertible,No Upfront,arn:aws:ec2:us-east-1:836060457634:reserved-in...


In [3]:
#  reservation DF

ri_fee = 'RIFee'

line_item_line_item_type = 'line_item_line_item_type'
reservation_reservation_a_r_n = 'reservation_reservation_a_r_n'
bill_billing_period_month = 'bill_billing_period_month'

effective_reservation_columns = ['bill_payer_account_id','line_item_usage_account_id','product_product_name',
'line_item_usage_type','line_item_line_item_type','reservation_reservation_a_r_n','line_item_unblended_rate',
'reservation_start_time','reservation_end_time','bill_billing_period_start_date','bill_billing_period_end_date',
'reservation_total_reserved_normalized_units','reservation_unused_normalized_unit_quantity',]

reservation_df = df.loc[:,effective_reservation_columns]


# getting reservation purchased
reservation_df = reservation_df[reservation_df[line_item_line_item_type] == ri_fee]



reservation_df['reservation_start_time'] = pd.to_datetime(reservation_df['reservation_start_time'])
reservation_df['reservation_end_time'] = pd.to_datetime(reservation_df['reservation_end_time'])

reservation_df['bill_billing_period_start_date'] = pd.to_datetime(reservation_df['bill_billing_period_start_date'])
reservation_df['bill_billing_period_end_date'] = pd.to_datetime(reservation_df['bill_billing_period_end_date'])

# Calculate the total hours between the start and end date
reservation_df['bill_billing_period_total_hours'] = (reservation_df['bill_billing_period_end_date'] - reservation_df['bill_billing_period_start_date']).dt.total_seconds() / 3600

# Calculate the total YEAR between the start and end date
reservation_df['reservation_purchase_term'] = ((reservation_df['reservation_end_time'] - reservation_df['reservation_start_time']).dt.days / 365).round()

# Calculate billing period month
reservation_df['bill_billing_period_month'] = (df['bill_billing_period_start_date'].dt.month)

# get columns 'pricing_offering_class', 'pricing_purchase_option' from discounted_usage_df
reservation_df = reservation_df.merge(discounted_usage_df, on='reservation_reservation_a_r_n', how='left')

# calculate reservation_utilization_cost,reservation_wastage_cost
reservation_df['reservation_utilization_cost'] = (reservation_df['reservation_total_reserved_normalized_units']).astype(float) * (reservation_df['line_item_unblended_rate']).astype(float)
reservation_df['reservation_wastage_cost'] = (reservation_df['reservation_unused_normalized_unit_quantity']).astype(float) * (reservation_df['line_item_unblended_rate']).astype(float)

# dorting data based on ['reservation_reservation_a_r_n','bill_billing_period_month']
reservation_df = reservation_df.sort_values(by=[reservation_reservation_a_r_n,reservation_reservation_a_r_n])



reservation_df.to_csv('reservation_utilization_wastage.csv', index=False)

reservation_df


,bill_payer_account_id,line_item_usage_account_id,product_product_name,line_item_usage_type,line_item_line_item_type,reservation_reservation_a_r_n,line_item_unblended_rate,reservation_start_time,reservation_end_time,bill_billing_period_start_date,bill_billing_period_end_date,reservation_total_reserved_normalized_units,reservation_unused_normalized_unit_quantity,bill_billing_period_total_hours,reservation_purchase_term,bill_billing_period_month,pricing_offering_class,pricing_purchase_option,reservation_utilization_cost,reservation_wastage_cost
5,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:t2.micro,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0083000000,2018-08-13 18:14:02+00:00,2019-08-13 18:14:01+00:00,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,372.00000000000,0.000000,744.0,1.0,12,convertible,No Upfront,3.08760,0.000000
14,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:t2.micro,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0083000000,2018-08-13 18:14:02+00:00,2019-08-13 18:14:01+00:00,2018-11-01 00:00:00+00:00,2018-12-01 00:00:00+00:00,360.00000000000,0.000000,720.0,1.0,11,convertible,No Upfront,2.98800,0.000000
23,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:t2.micro,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0083000000,2018-08-13 18:14:02+00:00,2019-08-13 18:14:01+00:00,2018-10-01 00:00:00+00:00,2018-11-01 00:00:00+00:00,372.00000000000,22.999722,744.0,1.0,10,convertible,No Upfront,3.08760,0.190898
8,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:t2.small,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0068000000,2018-04-03 01:22:45+00:00,2019-04-03 01:22:44+00:00,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,744.00000000000,0.000000,744.0,1.0,12,standard,Partial Upfront,5.05920,0.000000
17,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:t2.small,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0068000000,2018-04-03 01:22:45+00:00,2019-04-03 01:22:44+00:00,2018-11-01 00:00:00+00:00,2018-12-01 00:00:00+00:00,720.00000000000,0.000000,720.0,1.0,11,standard,Partial Upfront,4.89600,0.000000
25,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:t2.small,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0068000000,2018-04-03 01:22:45+00:00,2019-04-03 01:22:44+00:00,2018-10-01 00:00:00+00:00,2018-11-01 00:00:00+00:00,744.00000000000,0.000000,744.0,1.0,10,standard,Partial Upfront,5.05920,0.000000
1,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:c5.large,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0260000000,2018-04-03 01:21:13+00:00,2019-04-03 01:21:12+00:00,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,5952.00000000000,0.000000,744.0,1.0,12,standard,Partial Upfront,154.75200,0.000000
10,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:c5.large,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0260000000,2018-04-03 01:21:13+00:00,2019-04-03 01:21:12+00:00,2018-11-01 00:00:00+00:00,2018-12-01 00:00:00+00:00,5760.00000000000,0.000000,720.0,1.0,11,standard,Partial Upfront,149.76000,0.000000
19,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:c5.large,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0260000000,2018-04-03 01:21:13+00:00,2019-04-03 01:21:12+00:00,2018-10-01 00:00:00+00:00,2018-11-01 00:00:00+00:00,5952.00000000000,0.000000,744.0,1.0,10,standard,Partial Upfront,154.75200,0.000000
6,836060457634,836060457634,Amazon Elastic Compute Cloud,HeavyUsage:t2.micro,RIFee,arn:aws:ec2:us-east-1:836060457634:reserved-in...,0.0118000000,2018-10-09 21:40:47+00:00,2019-10-09 21:40:46+00:00,2018-12-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,372.00000000000,335.500000,744.0,1.0,12,standard,No Upfront,4.38960,3.958900
